H1 User Competency <> error ratio

In [ ]:
import pandas as pd
import pickle
from scipy import stats
import numpy as np
import seaborn as sns
import pingouin as pg
import statsmodels.formula.api as s

In [ ]:
#import and preprocess
df = pickle.load(open( '../00_data/cleaned/sitzungssummary.pkl','rb') )
xmlsaetze = pickle.load(open( '../00_data/cleaned/xmlsaetze.pkl','rb') )
final = pickle.load(open( '../00_data/final/competence.pkl','rb') )

final = final[['UserID','interventiongroup']]
rasch = xmlsaetze[['UserID','SatzID','Erfolg','Datum']]
rasch['UserID'].astype(str)
rasch['SatzID'].astype(str)

In [ ]:
# get and merge most recent task
most_recent_task = rasch.groupby(['UserID','SatzID']).agg({'Datum':np.max})
most_recent_task = most_recent_task.reset_index()
most_recent_task = pd.merge(most_recent_task, rasch, on=['UserID','SatzID','Datum'])

#drop rows that contain any value in the list
values = [9991775, 373 ,375]
most_recent_task = most_recent_task[most_recent_task.SatzID.isin(values) == False]

In [ ]:
df = pd.merge(most_recent_task,final, on='UserID')
df_grouped = df.groupby(['UserID','interventiongroup']).agg({'Erfolg':'sum','SatzID':'count'})
df_grouped['fehlerquotient'] = 1-(df_grouped.Erfolg / df_grouped.SatzID)

In [ ]:
# split by interventiongroup
final = df_grouped
final = final.reset_index()
intvcontrol = final[(final['interventiongroup']=='control')]
intv2 = final[(final['interventiongroup']=='2')]
intv3 = final[(final['interventiongroup']=='3')]
intv4 = final[(final['interventiongroup']=='4')]
intv5 = final[(final['interventiongroup']=='5')]
intv6 = final[(final['interventiongroup']=='6')]


In [ ]:
# get ratio of mistakes by group
prop_fehler_by_group = final.groupby(['interventiongroup']).agg({'fehlerquotient':'mean','UserID':'count'})
prop_fehler_by_group = prop_fehler_by_group.rename(columns={'UserID':'count'})
prop_fehler_by_group

In [ ]:
#Wilcoxon Mann Whitney Test
print(stats.mannwhitneyu(intvcontrol['fehlerquotient'], intv2['fehlerquotient']))
print(stats.mannwhitneyu(intvcontrol['fehlerquotient'], intv3['fehlerquotient']))
print(stats.mannwhitneyu(intvcontrol['fehlerquotient'], intv4['fehlerquotient']))
print(stats.mannwhitneyu(intvcontrol['fehlerquotient'], intv5['fehlerquotient']))
print(stats.mannwhitneyu(intvcontrol['fehlerquotient'], intv6['fehlerquotient']))
